## LAB9
Write a local-search algorithm (eg. an EA) able to solve the Problem instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:
 - Submission: Sunday, December 3 (CET)
 - Reviews: Sunday, December 10 (CET)

Notes:
 - Reviews will be assigned on Monday, December 4
 - You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [284]:
from random import choice, choices, random
from copy import deepcopy

import lab9_lib

In [285]:
GENERATIONS = 10000
INDIVIDUAL_SIZE = 1000
POPULATION_SIZE = 100
ACCEPTABLE_FITNESS = 0.9

In [286]:
def create_population(number_of_indivinuals, individual_size):
    population = []
    for _ in range(number_of_indivinuals):
        population.append(choices([0, 1], k=individual_size))
    return population


# return the population ordered by fitness, the return is a list of tuples [individual, fitness]
def fitness_check(population_with_fitness : list, new_population, fitness_function, num_survivors):

    for individual in new_population:
        population_with_fitness.append([individual, fitness_function(individual)])

    population_with_fitness.sort(key=lambda x: x[1], reverse=True)
    return population_with_fitness[0 : num_survivors]


def repopulate(population, new_generation_size, best_fitness):
    offsprings = []
    mutation_rate = 0.5
    max_mutation_size = 100
    crossover_parents = 10
    crossover_parent_inheritance = int(len(population[0]) / crossover_parents)

    for _ in range(new_generation_size):
        parents = choices(population, k=crossover_parents)
        offspring = []
        for i, parent in enumerate(parents):
            x = i * crossover_parent_inheritance
            offspring += parent[x : x + crossover_parent_inheritance]
        offsprings.append(offspring)
    
    for offspring in offsprings:
        if random() < mutation_rate:
            mutation_size = choice(range(max_mutation_size))
            for _ in range(mutation_size):
                g = choice(range(len(offspring)))
                offspring[g] = 1 - offspring[g]

    return offsprings
        

In [287]:
def ea(fitness, problem_size : int, pop_size : int, ind_size : int):
    num_survivors = int(pop_size / 10)
    population = create_population(num_survivors, ind_size)
    fitted_population = fitness_check([], population, fitness, num_survivors)
    best_fitness = fitted_population[0][1]
    stagnation_counter = GENERATIONS / 20

    for gen in range(GENERATIONS):
        
        pop_copy = [ind[0] for ind in fitted_population]
        offsprings = repopulate(pop_copy, pop_size-num_survivors, best_fitness)

        fitted_population = fitness_check(fitted_population, offsprings, fitness, num_survivors)
        new_best_fitness = fitted_population[0][1]

        if best_fitness < new_best_fitness:
            print(f"Problem size {problem_size}, Generation {gen+1} : {new_best_fitness:.2%}")
            best_fitness = new_best_fitness
            stagnation_counter = GENERATIONS / 100
        else:
            if best_fitness >= ACCEPTABLE_FITNESS:
                stagnation_counter -= 1
                if stagnation_counter == 0:
                    return population[0], best_fitness

        if best_fitness == 1:
            return population[0], best_fitness
        
        '''if best_fitness >= ACCEPTABLE_FITNESS:
            stagnation_counter -= 1
            if stagnation_counter == 0:
                return population[0], best_fitness'''
        
    
    return population[0], best_fitness

In [288]:
def mutate(individual, mutation_rate):
    for gene in range(len(individual)):
        if random() < mutation_rate:
            individual[gene] = 1
        else:
            individual[gene] = 0
    return individual

def ea2(fitness, problem_size : int, pop_size : int, ind_size : int):
    
    mutation_rate = 0.5
    population = create_population(pop_size, ind_size)
    population.sort(key=fitness, reverse=True)
    best_fitness = fitness(population[0])
    breakout_counter = GENERATIONS / 10

    crossover_parents = 2
    num_offsprings = 2
    crossover_parent_inheritance = int(len(population[0]) / crossover_parents)

    for gen in range(GENERATIONS):
        parents = choices(population, k=crossover_parents, weights=[fitness(individual) for individual in population])

        offsprings = []
        for off in range(num_offsprings):
            offspring = []
            for i, parent in enumerate(parents):
                x = i * crossover_parent_inheritance
                offspring += parent[x : x + crossover_parent_inheritance]
            offsprings.append(mutate(offspring, mutation_rate))

        population[pop_size - 1] = offsprings[0]
        population[pop_size - 2] = offsprings[1]
        population.sort(key=fitness, reverse=True)

        current_best_fitness = fitness(population[0])

        print(f"Problem size {problem_size}, Generation {gen+1} : {current_best_fitness:.2%}")

        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            mutation_rate = mutation_rate * 0.9
        else:
            mutation_rate = mutation_rate * 1.1

        if best_fitness == 1:
            return population[0], best_fitness
        
        if best_fitness >= ACCEPTABLE_FITNESS:
            breakout_counter -= 1
            if breakout_counter == 0:
                return population[0], best_fitness
    
    return population[0], best_fitness

In [289]:
results = []
for problem_size in [1, 2, 5, 10]:
    fitness_function = lab9_lib.make_problem(problem_size)
    individual, individual_fitness = ea(fitness_function, problem_size, POPULATION_SIZE, INDIVIDUAL_SIZE)
    print(f'Final result : {individual_fitness:.2%}')
    # print(f"{''.join(str(i) for i in individual)} : {fitness_function(individual):.2%}")
    calls = fitness_function.calls
    print(f'fitness calls: {calls}')
    results.append([problem_size, individual_fitness, calls])

Problem size 1, Generation 1 : 54.90%
Problem size 1, Generation 2 : 55.80%
Problem size 1, Generation 3 : 57.00%
Problem size 1, Generation 4 : 58.00%
Problem size 1, Generation 5 : 58.80%
Problem size 1, Generation 6 : 59.50%
Problem size 1, Generation 7 : 60.10%
Problem size 1, Generation 8 : 60.90%
Problem size 1, Generation 9 : 61.20%
Problem size 1, Generation 10 : 61.50%
Problem size 1, Generation 11 : 61.70%
Problem size 1, Generation 12 : 62.20%
Problem size 1, Generation 13 : 62.60%
Problem size 1, Generation 14 : 63.50%
Problem size 1, Generation 15 : 63.70%
Problem size 1, Generation 16 : 64.50%
Problem size 1, Generation 17 : 64.90%
Problem size 1, Generation 18 : 65.40%
Problem size 1, Generation 19 : 65.70%
Problem size 1, Generation 20 : 66.10%
Problem size 1, Generation 21 : 66.70%
Problem size 1, Generation 22 : 66.90%
Problem size 1, Generation 23 : 67.60%
Problem size 1, Generation 24 : 67.70%
Problem size 1, Generation 25 : 68.10%
Problem size 1, Generation 26 : 68

In [290]:
for result in results:
    print(f'Problen size: {result[0]}')
    print(f'Best fitness: {result[1]:.2%}')
    print(f'Fitness calls: {result[2]}')
    print()

Problen size: 1
Best fitness: 95.70%
Fitness calls: 163180

Problen size: 2
Best fitness: 90.00%
Fitness calls: 628480

Problen size: 5
Best fitness: 54.00%
Fitness calls: 900010

Problen size: 10
Best fitness: 35.93%
Fitness calls: 900010

